In [1]:
import dataiku
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Here data should be loaded etc.

# Weather data (KNMI)

Source: https://www.knmi.nl/nederland-nu/klimatologie/uurgegevens

**Column Meaning**

**YYYYMMDD**  = datum (YYYY=jaar,MM=maand,DD=dag) / date (YYYY=year,MM=month,DD=day) <br>
**HH**        = Time (HH uur/hour, UT. 12 UT=13 MET, 14 MEZT. Hourly division 05 runs from 04.00 UT to 5.00 UT <br>
**DD**        = Mean wind direction (in degrees) during the 10-minute period preceding the time of observation (360=north, 90=east, 180=south, 270=west, 0=calm 990=variable) <br>
**FH**        = Hourly mean wind speed (in 0.1 m/s) <br>
**FF**        = Mean wind speed (in 0.1 m/s) during the 10-minute period preceding the time of observation <br>
**FX**        = Maximum wind gust (in 0.1 m/s) during the hourly division <br>
**T**        = Temperature (in 0.1 degrees Celsius) at 1.50 m at the time of observation <br>
**T10N**      = Minimum temperature (in 0.1 degrees Celsius) at 0.1 m inthe preceding 6-hour period <br>
**TD**        = Dew point temperature (in 0.1 degrees Celsius) at 1.50 m at the time of observation <br>
**SQ**        = Sunshine duration (in 0.1 hour) during the hourly division, calculated from global radiation (-1 for <0.05 hour) <br>
**Q**        = Global radiation (in J/cm2) during the hourly division <br>
**DR**        = Precipitation duration (in 0.1 hour) during the hourly division<br>
**RH**        = Hourly precipitation amount (in 0.1 mm) (-1 for <0.05 mm) <br>
**P**        = Air pressure (in 0.1 hPa) reduced to mean sea level, at the time of observation <br>
**VV**        = Horizontal visibility at the time of observation (0=less than 100m, 1=100-200m, 2=200-300m,..., 49=4900-5000m, 50=5-6km, 56=6-7km, 57=7-8km, ..., 79=29-30km, 80=30-35km, 81=35-40km,..., 89=more than 70km)<br>
**N**        = Cloud cover (in octants), at the time of observation (9=sky invisible) <br>
**U**        = Relative atmospheric humidity (in percents) at 1.50 m at the time of observation <br>
**WW**        = Present weather code (00-99), description for the hourly division. <br>
**IX**        = Indicator present weather code (1=manned and recorded (using code from visual observations), 2,3=manned and omitted (no significant weather phenomenon to report, not available), 4=automatically recorded (using code from visual observations), 5,6=automatically omitted (no significant weather phenomenon to report, not available), 7=automatically set (using code from automated observations) <br>
**M**        = Fog 0=no occurrence, 1=occurred during the preceding hour and/or at the time of observation <br>
**R**        = Rainfall 0=no occurrence, 1=occurred duringthe preceding hour and/or at the time of observation <br>
**S**        = Snow 0=no occurrence, 1=occurred during the preceding hour and/or at the time of observation <br>
**O**        = Thunder  0=no occurrence, 1=occurred during the preceding hour and/or at the time of observation <br>
**Y**        = Ice formation 0=no occurrence, 1=occurred during the preceding hour and/or at the time of observation <br>

In [9]:
knmi_df['YYYYMMDD'] = pd.to_datetime(knmi_df['YYYYMMDD'], format='%Y%m%d')

In [10]:
knmi_df_day = knmi_df.groupby(['YYYYMMDD']).agg(temp=('T','mean'), humidity=('U', 'mean')).reset_index()

# convert from 0.1 degrees to degrees
knmi_df_day['temp'] = knmi_df_day['temp'] / 10

Unused columns should be removed here

# Filter Potassium

In [12]:
# first drop rows where Potassium is missing
df.dropna(subset=['pos_level'], inplace=True)

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(data=df, x="pos_level", kde=True, stat='percent')
plt.show()

In [ ]:
# Look at the odd values below 0.4
df[df['pos_level'] < 0.4]

In [16]:
df.drop(df[df['pos_level'] < 0.4].index  , inplace=True)

In [17]:
df.reset_index(drop=True, inplace=True)

# EDA

**Boxplot feature against Potassium**

In [18]:
def boxplot_feature(df, feature, display_samples = False):
    '''
    Displays a boxplot of the selected feature per each production run and also per dataset (all production runs).

    * If display_samples is True, then the full dataset plot will also include all the point samples from each group.
    * bin_type refers to the binning method:
        - unifo
    '''
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16,10))
    
    # per production run
    bp = sns.boxplot(ax = ax[0], x="Production_run", y="pos_level", data=df, showmeans=True, hue=feature,
               meanprops={"marker": "_", "markeredgecolor": "red", "markersize": "20", 'markeredgewidth':'3' })
    ax[0].set(title=f'Potassium content against {feature} for each Production run', ylabel='Potassium')

    # for all the dataset
    bp = sns.boxplot(ax = ax[1], x=feature, y="pos_level", data=df, showmeans=True,
                     meanprops={"marker": "_", "markeredgecolor": "red", "markersize": "20", 'markeredgewidth':'3' })
    ax[1].set(title=f'Potassium content against {feature} for all data', ylabel='Potassium')
    if display_samples:
        ax = sns.swarmplot(x=feature, y="pos_level", data=df, color=".25")
        
    fig.suptitle(f'Influence of {feature} on Potassium (Red line shows the average)', fontsize = 16)

    fig.tight_layout()
    plt.show()

**We will use an exploratory dataframe from now on**

In [40]:
dfe = df.copy()

## PR_cursor

This shows for each production run where the sample is in the production. Low numbers are the first produced batches, higher numbers are the later produced batches

In [19]:
dfe['PR_cursor'] = 1
dfe['PR_cursor'] = dfe[['Production_run','PR_cursor']].groupby(['Production_run']).cumsum()

In [ ]:
dfe['PR_cursor'].value_counts().plot.bar(figsize=(14,4));

#### Bin PR_cursor - Boxplot

In [21]:
# PRc = PR_cursor
uniform_bins_PRc = [0, 5, 10, 20, 30, 100]
custom_bins_PRc = [0, 5, 25, 100]
perc_bins_PRc = [0, 0.3, 0.7, 1]

# create 'percentage' binning
pr_max = dfe[['Production_run', 'Production_pallet']].groupby('Production_run').transform('max')
dfe['perc_bin_PRc'] = dfe['Production_pallet'] / pr_max['Production_pallet']

# create binned columns
dfe['uni_bin_PRc'] = pd.cut(dfe['PR_cursor'], bins=uniform_bins_PRc)
dfe['custom_bin_PRc'] = pd.cut(dfe['PR_cursor'], bins=custom_bins_PRc)
dfe['perc_bin_PRc'] = pd.cut(dfe['perc_bin_PRc'], bins=perc_bins_PRc)

In [ ]:
binned_features = ['uni_bin_PRc', 'custom_bin_PRc', 'perc_bin_PRc']

for feat in binned_features:
    boxplot_feature(dfe, feat)

The binned PR cursor features don't give a clear result, so these will be removed.

In [23]:
# This feature didn't gave a good result, so bins are deleted
dfe.drop(columns = ['uni_bin_PRc', 'custom_bin_PRc', 'perc_bin_PRc'], inplace=True)

## Potassium distribution per production run

In [24]:
dfe_production = dfe.pivot("PR_cursor", "Production_run", "pos_level")

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe_production)
plt.show()

In [ ]:
 boxplot_feature(dfe, "Production_run", display_samples = False)

## Sensor features

### Milk flow

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(data=dfe, x="milk_flow", kde=True, stat='percent')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe, x="potasium", y="milk_flow")
plt.show()

In [ ]:
# Look at values that are very off
dfe.loc[(dfe["milk_flow"] < 100) | (dfe["milk_flow"] > 4000)]

In [30]:
dfe.drop(dfe[(dfe["milk_flow"] < 100) | (dfe["milk_flow"] > 4000)].index, inplace=True)
dfe.reset_index(drop=True, inplace=True)

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe, x="potasium", y="milk_flow")
plt.show()

In [32]:
# bin the milk flow content
custom_milk_bins = [0, 2000, 2500, 3000]

dfe['binned_milk_flow'] = pd.cut(dfe['milk_flow'], bins=custom_milk_bins)

In [ ]:
boxplot_feature(dfe, "binned_milk_flow")

This could be a bit of an influence, so we will keep this feature. With more milk the average is a bit higher, but it's a minor difference.

### Whey flow

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(data=dfe, x="whey_flow", kde=True, stat='percent')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe, x="potasium", y="whey_flow")
plt.show()

In [ ]:
# Look at odd values
dfe.loc[(dfe["whey_flow"] < 1000)]

In [37]:
dfe.drop(dfe[dfe["whey_flow"] < 1000].index, inplace=True)
dfe.reset_index(drop=True, inplace=True)

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe, x="potasium", y="whey_flow")
plt.show()

In [39]:
# bin the lactose content
custom_whey_bins = [0, 3000, 4000, 5000, 6000]

dfe['binned_whey_flow'] = pd.cut(dfe['whey_flow'], bins=custom_whey_bins)

In [ ]:
boxplot_feature(dfe, "binned_whey_flow")

Will keep this feature, can be a correlation

### Milk whey ratio

In [41]:
dfe["milk_whey_flow_ratio"] = dfe.apply(lambda x: x.milk_flow / (x.milk_flow + x.whey_flow) * 100, axis = 1)

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(data=dfe, x="milk_whey_flow_ratio", kde=True, stat='percent')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))

sns.lineplot(data=dfe, x="potasium", y="milk_whey_flow_ratio")
plt.show()

In [44]:
custom_milk_whey_bins = [0, 25, 32, 40, 50]

dfe["binned_whey_milk"] = pd.cut(dfe["milk_whey_flow_ratio"], bins=custom_milk_whey_bins) 

In [ ]:
boxplot_feature(dfe, "binned_whey_milk")

There is not a clear correlation between potassium levels and the milk whey flow ratio

### Lactose

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(data=dfe, x="lactose", kde=True, stat='percent')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe, x="potasium", y="lactose")
plt.show()

In [ ]:
dfe.loc[dfe["lactose"] < 400]

In [50]:
dfe.drop(dfe[dfe["lactose"] < 400].index, inplace=True)
dfe.reset_index(drop=True, inplace=True)

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe, x="potasium", y="lactose")
plt.show()

In [52]:
# bin the lactose content
custom_lactose_bins = [0, 650, 800, 1000]

dfe['binned_lactose'] = pd.cut(dfe['lactose'], bins=custom_lactose_bins)

In [ ]:
boxplot_feature(dfe, 'binned_lactose')

Would conclude out of this that there is a correlation between the amount of lactose and the amount of potassium. When there is a lot lactose there is less potassium.

### Fat flow

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(data=dfe, x="fat_flow", kde=True, stat='percent')
plt.show()

There are some outliers in this data, so these values are removed. They are probably invalid values.

In [ ]:
fig = plt.figure(figsize=(14,4))

sns.lineplot(data=dfe, x="potassium", y="fat_flow")
plt.show()

In [ ]:
dfe.loc[dfe["fat_flow"] < 750]

In [57]:
dfe.drop(dfe[dfe["fat_flow"] < 750].index, inplace=True)
dfe.reset_index(drop=True, inplace=True)

In [ ]:
fig = plt.figure(figsize=(14,4))

sns.lineplot(data=dfe, x="potasium", y="fat_flow")
plt.show()

In [ ]:
fat_bins = [0, 1250, 1500, 1750, 2000]
dfe['binned_fat_flow'] = pd.cut(dfe['fat_flow'], bins=fat_bins)
boxplot_feature(dfe, 'binned_fat_flow')

This gives that high fat results in low potassium levels, this could be relevant so we keep these features.

### Added PotassiumChloride

In [60]:
# Increasing the value, they were too small to see good realtions
dfe['KCl'] *= 100000

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(data=dfe, x="KCl", kde=True, stat='percent')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))

sns.lineplot(data=dfe, x="potasium", y="KCl")
plt.show()

In [ ]:
dfe.loc[dfe["KCl"] == 0]

From production 18 there is no Potassium Chloride added anymore, this could influence the model depending on where the split for training and test data is.

In [ ]:
added_KCl_bins = [0, 3000, 3500, 4000, 6000]
dfe['binned_KCl'] = pd.cut(dfe['KCl'], bins=added_KCl_bins)
boxplot_feature(dfe, 'binned_KCl')

This shows that more PotassiumChloride added, gives also more potassium in the product. But values are very close to each other, so how much influence is it really?

In [65]:
dfe.drop(columns = ["binned_KCl"], inplace=True)

### Milk Protein

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(data = dfe, x="protein", kde=True, stat='percent')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))

sns.lineplot(data=dfe, x="potasium", y="protein")
plt.show()

In [ ]:
dfe.loc[dfe["protein"] < 3]

In [69]:
dfe.drop(dfe[dfe["protein"] < 3].index, inplace=True)
dfe.reset_index(drop=True, inplace=True)

In [ ]:
fig = plt.figure(figsize=(14,4))

sns.lineplot(data=dfe, x="potasium", y="protein")
plt.show()

In [ ]:
milk_pro_bins = [0, 3.5, 3.75, 4]
dfe['binned_protein'] = pd.cut(dfe['protein'], bins=milk_pro_bins)
boxplot_feature(dfe, 'binned_protein')

More protein in the milk results in less potassium in the end product

## KNMI features

### Temperature (KNMI)

In [72]:
# add the Temperature to the original dataset
dfe = pd.merge(dfe, knmi_df_day[['YYYYMMDD', 'temp']], how='left', left_on=['production_date'], right_on=['YYYYMMDD']) \
        .drop(columns=['YYYYMMDD'])

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(data=dfe, x="temp", kde=True, stat='percent', bins=10)
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe, x="potassium", y="temp")
plt.show()

In [75]:
# bin temperatures with custom bins - I tried to go with very cold, cold, cold-neutral, warm-neutral, warm, very hot
uniform_bins_temp = [-10, 0, 10, 20, 30]

dfe['uni_binned_temp'] = pd.cut(dfe['temp'], bins=uniform_bins_temp)
dfe['q_binned_temp'] = pd.qcut(dfe['temp'], q=4)

In [ ]:
temp_bins = ['uni_binned_temp', 'q_binned_temp']

for feat in temp_bins:
    boxplot_feature(dfe, feat, display_samples = False)

If you look at uni_binned_temp, there it shows that a lower temperature has a lower potassium level. q_binned_temp doesn't give a good result so this one will be removed.

In [77]:
dfe.drop(columns = ["q_binned_temp"], inplace=True)

### Month of the year

In [41]:
# Jan = 1, December = 12
dfe['month'] = dfe['production_date'].dt.month

# convert to March = 1 (starts to get warm) to February = 12 (end of cold season)
dfe['month'] = dfe['month'] - 2

# take care of negative months: January = -1 -> 11, February = 0 -> 12
dfe.loc[dfe['month'] == -1, 'month'] = 11
dfe.loc[dfe['month'] == 0, 'month'] = 12

In [42]:
# consider also binning by seasons
seasons = [0, 3, 6, 9 , 12]

dfe['binned_month']  = pd.cut(dfe['month'], bins=seasons, labels=['warm', 'hot', 'cold','freezing'])

In [ ]:
boxplot_feature(dfe, 'month', display_samples = True)

Month 12 has relative low potassium levels, in this month only temperatures below zero are measured. So this is kind of the same explaination as the temperature discovery. 
Month 6 has relative high potassium levels, in this month the temperature was around 20 degrees. From the temperature feature it would be that this also gives high potassium levels. 
Could also be that in months with lower temperatures maybe more lactose is added, which dilutes the potassium levels.

### Relative Atmosferic Humidity (KNMI)

In [81]:
# add the Humidity to the original dataset
dfe = pd.merge(dfe, knmi_df_day[['YYYYMMDD', 'humidity']], how='left', left_on=['production_date'], right_on=['YYYYMMDD']) \
        .drop(columns=['YYYYMMDD'])

In [ ]:
fig = plt.figure(figsize=(14,4))

custom_humidity = dfe['humidity']
sns.histplot(x=custom_humidity, kde=True, stat='percent')

N = 100
data = plt.np.linspace(50, N, 21)

plt.xticks(data) # add loads of ticks
plt.grid()
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))

sns.lineplot(data=dfe, x="potassium", y="humidity")
plt.show()

In [ ]:
# bin humidity
humidity_bins = [0, 75, 82, 87, 90, 100]
dfe['binned_humidity'] = pd.cut(dfe['humidity'], bins=humidity_bins)

boxplot_feature(dfe, 'binned_humidity')

Will keep the humidity, because a low humidity seems to result in a low potassium.

### Rolling window Humidity

In [85]:
NDAYS = 10

# create the average temperature over the last 10 days
dfe['rol_humidity'] = dfe['humidity'].rolling(NDAYS).mean()

In [86]:
# ONLY FOR PLOTTING
# get rid of NaNs
dfe_plot = dfe.dropna(how='any')

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe_plot, x="potassium", y="rol_humidity")
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(x=dfe['rol_humidity'], kde=True, stat='percent')
plt.show()

In [89]:
# bin temperatures with custom bins - I tried to go with very cold, cold, cold-neutral, warm-neutral, warm, very hot
bins_rol_humidity = [50, 75, 80, 87, 90, 100]
dfe['binned_rol_humidity'] = pd.cut(dfe['rol_humidity'], bins=bins_rol_humidity)

In [ ]:
boxplot_feature(dfe, 'binned_rol_humidity')

This shows that a lower humidity over the past 10 days, could give lower potassium levels.

### Rolling window Temperature

In [91]:
NDAYS = 10

# create the average temperature over the last 10 days
dfe['rol_temp'] = dfe['temp'].rolling(NDAYS).mean()

In [92]:
# ONLY FOR PLOTTING
# get rid of NaNs
dfe_plot = dfe.dropna(how='any')

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.lineplot(data=dfe_plot, x="potassium", y="rol_temp")
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,4))
sns.histplot(x=dfe['rol_temp'], kde=True, stat='percent')
plt.show()

In [95]:
# bin temperatures with custom bins - I tried to go with very cold, cold, cold-neutral, warm-neutral, warm, very hot
bins_rol_temp = [-10, 0, 10, 20, 30]
dfe['binned_rol_temp'] = pd.cut(dfe['rol_temp'], bins=bins_rol_temp)

In [ ]:
boxplot_feature(dfe, 'binned_rol_temp')

# Add data to original dataframe

In [19]:
# Temperature
df = pd.merge(df, knmi_df_day[['YYYYMMDD', 'temp']], how='left', left_on=['production_date'], right_on=['YYYYMMDD']) \
        .drop(columns=['YYYYMMDD'])

In [20]:
NDAYS = 10
# create the average temperature over the last 10 days
df['rol_temp'] = df['temp'].rolling(NDAYS).mean()

In [21]:
# Humidity
df = pd.merge(df, knmi_df_day[['YYYYMMDD', 'humidity']], how='left', left_on=['production_date'], right_on=['YYYYMMDD']) \
        .drop(columns=['YYYYMMDD'])

In [22]:
df['rol_humidity'] = df['humidity'].rolling(NDAYS).mean()

In [23]:
# seasonal month

# Jan = 1, December = 12
df['month'] = df['month'].dt.month

# convert to March = 1 (starts to get warm) to February = 12 (end of cold season)
df['month'] = df['month'] - 2

# take care of negative months: January = -1 -> 11, February = 0 -> 12
df.loc[df['month'] == -1, 'month'] = 11
df.loc[df['month'] == 0, 'month'] = 12

# Feature Correlation with Target Variable

Drop all unused columns here

In [25]:
df_corr = df.corr()['potassium'].sort_values(ascending=False)[1:].dropna()

In [ ]:
plt.title("Correlation of each feature with potassium")
df_corr.plot.barh(figsize=(10,8));

In [31]:
corr_df = df[["potassium", "lactose", "month", "fat", "protein", "KCl", "whey_flow", "milk_flow", "rol_humidity", "rol_temp"]]

In [34]:
corr_df = corr_df.corr()
abs_corr_df = corr_df.abs()

In [ ]:
sns.set(rc = {'figure.figsize':(9,8)})
plt.title("Correlation graph between features")
sns.heatmap(abs_corr_df)